<a href="https://colab.research.google.com/github/ea-Mitsuoka/ionicons/blob/master/bqml_recommend_anime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
```
Copyright 2019 Google LLC

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
```
---

# Matrix Factorization デモ

BigQuery ML は Matrix Factorization を alpha 機能として追加しました。これは一般的に、商品、映画、アプリなどの推薦システムに使用される行列分解の手法で、一種の協調フィルタリングアルゴリズムです。 協調フィルタリングは、目に見えないアイテムのうちどれがユーザーにとって関心があるかを予測するため、ユーザーの既存のアイテム評価から学習を行います。
詳細はこちらの[ブログ](https://medium.com/google-cloud-jp/bigqueryml-recommendation-3c6cbc2f64ea)も参照ください。

In [ ]:
import pandas as pd
import numpy as np
import sys
from google.cloud import bigquery
from google.colab import auth
auth.authenticate_user()
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#@title プロジェクト変数の設定 { run: "auto", display-mode: "form" }
project_id = '' #@param {type:"string"}
#dataset_name = "cycle" #@param {type:"string"}
#eval_name = model_name + "_eval"
client = bigquery.Client(project=project_id)

ここでは、[Anime Recommendations Database](https://www.kaggle.com/CooperUnion/anime-recommendations-database/downloads/anime-recommendations-database.zip/1) のサンプルデータを使って Recommendation モデルを作成してみます。

## モデル作成

入力データにはユーザー、アイテム、評価の3列を必要とします。が含まれます。また、以下のような固有のモデルオプションがあります。

* num_factors（INT）：ユーザー項目の潜在的な要素の数。デフォルトでは、round（log_2（num_rows））に設定されています。最大num_factorsは200です。
* user_col（STRING）：ユーザーID
* item_col（STRING）：商品ID
* rating_col（STRING）：ユーザーごとのアイテムへの評価。任意の数値型にすることができます。

さらに、以下を除くすべてのモデルオプションがサポートされています。
* L1_REG
* INPUT_LABEL_COLS
* CLASS_WEIGHTS
* AUTO_CLASS_WEIGHTS
* NUM_CLUSTERS
* DISTANCE_TYPE

「Model is too large (>100MB)」エラーが発生した場合、入力データを確認してください。これは、単一ユーザーまたは単一アイテムに対して評価が多すぎることが原因です。ユーザーまたはアイテムの列をINT64にハッシュするか、データサイズを小さくして対処します。

In [ ]:
query = """
CREATE OR REPLACE MODEL anime.recommend
OPTIONS (model_type='matrix_factorization', user_col='user_id', item_col='anime_id',rating_col='rating', l2_reg=9.83, num_factors=40) AS 
SELECT *
FROM anime.data_clean
--WHERE anime_id in
--(SELECT anime_id FROM anime.anime where members >10000)
"""
df = client.query(query).to_dataframe()
df

## トレーニング情報、モデル評価
モデルに対して、ML.TRAINING_INFOは以下を返します。
* Training_run
* Iteration
* Duration_ms
* Loss（トレーニングデータの損失）
* Eval_loss（テストデータの損失）

このステートメントをモデルのトレーニング中に実行して、最新のiterationまでの統計を取得できます。 統計は、BigQuery UIのモデル詳細にも表示されます。

In [ ]:
query = """
SELECT * FROM ML.TRAINING_INFO(MODEL anime.recommend)
"""
df = client.query(query).to_dataframe()
df

,training_run,iteration,loss,eval_loss,duration_ms
0,0,2,1.104536,1.416501,137219
1,0,1,1.391246,1.473982,625158
2,0,0,56.235097,844.675661,747287


ML.EVALUATEは、線形回帰モデルのML.EVALUATEと同じメトリックを返します。
メトリックは、ml.EVALUATEで指定された入力データに基づいて計算されます。

In [ ]:
query = """
SELECT * from ML.EVALUATE(MODEL anime.recommend)
"""
df = client.query(query).to_dataframe()
df

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,0.887628,1.416501,0.026521,0.687446,0.430176,0.430176


## 予測
特定のユーザーとアイテムのペアに関連付けられた評価を表示するには、モデル名と共にML.PREDICTを使用します。 出力は、各ユーザーとアイテムのペアの評価を返します。

In [ ]:
query = """
SELECT *
FROM ML.PREDICT(MODEL anime.recommend, (
  SELECT * FROM anime.data_clean LIMIT 5
  ))
"""
df = client.query(query).to_dataframe()
df


,predicted_rating,user_id,anime_id,rating
0,7.276951,7670,4188,7
1,7.550864,7670,4454,7
2,6.658817,7670,5484,7
3,7.402746,7670,5923,7
4,6.144292,7670,6201,7


## レコメンデーション
ML.RECOMMENDは、ユーザーとアイテムのペアごとに評価を生成します。
この結果ははかなり大きな出力になる可能性があるため、出力はテーブルに保存してから分析のために他のクエリで使用することをお勧めします。

In [ ]:
query = """
#自分と好みが似ているユーザー
SELECT d.user_id, count(a.name)
FROM anime.data_clean d,anime.anime a
WHERE a.anime_id=d.anime_id
AND d.user_id in (
SELECT user_id
FROM anime.data_clean d
WHERE anime_id in (1,513,523,3572,9253,9756,10721,15227,11757)
GROUP BY d.user_id
HAVING count(anime_id) > 7
--ORDER BY 2 desc
)
GROUP BY 1
ORDER BY 2 ASC
"""
df = client.query(query).to_dataframe()
df

,user_id,f0_
0,22243,143
1,4391,164
2,66819,219
3,54327,225
4,1811,228
5,57987,243
6,19241,262
7,44474,265
8,24890,285
9,13362,290


In [ ]:
query = """
SELECT a.name,d.rating
FROM anime.data_clean d,anime.anime a
WHERE a.anime_id=d.anime_id
AND d.user_id = 22243
--GROUP BY 1
ORDER BY 2 DESC
"""
df = client.query(query).to_dataframe()
df

,name,rating
0,Suzumiya Haruhi no Yuuutsu,10
1,Serial Experiments Lain,10
2,Suzumiya Haruhi no Yuuutsu (2009),10
3,Lucky☆Star,10
4,Suzumiya Haruhi no Shoushitsu,10
5,Lucky☆Star: Original na Visual to Animation,10
6,Kaze no Tani no Nausicaä,10
7,Tonari no Totoro,9
8,Summer Wars,9
9,Full Metal Panic? Fumoffu,9


In [ ]:
query = """
SELECT a.name,r.predicted_rating
 FROM ML.RECOMMEND(MODEL anime.recommend) r
 ,anime.anime a 
 WHERE a.anime_id=r.anime_id 
 AND r.user_id=22243
 ORDER BY predicted_rating desc
"""
df = client.query(query).to_dataframe()
df

,name,predicted_rating
0,Ginga Eiyuu Densetsu,9.496288
1,Kimi no Na wa.,9.171726
2,Steins;Gate,9.163599
3,Gintama°,9.040414
4,Hellsing Ultimate,8.948962
5,Mushishi,8.939329
6,Steins;Gate Movie: Fuka Ryouiki no Déjà vu,8.929245
7,Gintama,8.910870
8,Fullmetal Alchemist: Brotherhood,8.905807
9,Rurouni Kenshin: Meiji Kenkaku Romantan - Tsui...,8.903969
